In [30]:
import csv
import re
import os
import json
import requests
import asyncio
import aiohttp
from datetime import datetime
from dotenv import load_dotenv
import random
import nest_asyncio
import pandas as pd
import numpy as np

In [31]:
# Reading the csv generated after resolving multiple addresses returned by web-scraping code against airport proximity.
# And selecting one address basis proximity to airport.
input_file_addresses = "/Users/shreya/Downloads/Returned_address_Web_scraping_OneAddressSelected.csv"
web_scraped_address = pd.read_csv(input_file_addresses)

In [32]:
print(web_scraped_address.head(2))

                     agency_name  \
0  Billings Clinic Cancer Center   
1      Moores UCSD Cancer Center   

                                             website  \
0  https://www.billingsclinic.com/maps-locations/...   
1  https://healthlocations.ucsd.edu/san-diego/385...   

                                             address  \
0  [{'addresses': []}, {'addresses': {'addresses'...   
1  [{'addresses': [{'addressLine1': '3855 Health ...   

                       shortest_distance_address  
0            801 North 29th St Billings MT 59101  
1  3855 Health Sciences Drive San Diego CA 92037  


In [ ]:
# Reading the csv generated by Peilu of the 100 most flight-requested agencies from agency table, for flights which have a requester
# This is the base csv. This is what was passed through the Serper API code, and then Playwright code
input_file_agencies = "/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/Peilu_List_Agencies/100 most requested-2.csv"
afw_agency_address = pd.read_csv(input_file_agencies)

In [34]:
afw_agency_address = afw_agency_address.replace(np.nan, '', regex = True)
full_agency_address = afw_agency_address["address1"] + " " +\
                        afw_agency_address["address2"] + " " +\
                        afw_agency_address["city"] + " " +\
                        afw_agency_address["state"] + " " +\
                        afw_agency_address["zipcode"]
afw_agency_address["full_agency_address"] = full_agency_address

In [35]:
print(afw_agency_address.head(2))

     id                         agency_name             address1 address2  \
0  4778  Lucile Packard Children's Hospital         725 Welch Rd            
1  2297       Stanford Hospital and Clinics  900 Blake Wilbur Dr            

  site department       city state     zipcode               created_at  \
0                  Palo Alto    CA  94304-1601  2023-09-05 22:14:59.000   
1                  Palo Alto    CA  94304-2201  2024-02-06 21:02:15.000   

   year_creation  flight_count                           full_agency_address  
0           2023           201         725 Welch Rd  Palo Alto CA 94304-1601  
1           2024            83  900 Blake Wilbur Dr  Palo Alto CA 94304-2201  


In [36]:
def fetch_web_scraped_address(agency_name):
    if agency_name in web_scraped_address["agency_name"].values: 
        return web_scraped_address.loc[web_scraped_address["agency_name"] == agency_name, "shortest_distance_address"].iloc[0]
    else: return "NA"

In [37]:
afw_agency_address.assign(web_scraped_address = "NA")
for _, each_agency in afw_agency_address.iterrows():
    shortest_distance_address = fetch_web_scraped_address(each_agency["agency_name"])
    afw_agency_address.loc[_, ["web_scraped_address"]] = shortest_distance_address

In [38]:
# Reading the csv generated by Serper API code
# And fetching address_from_search_snippet, address_from_search_answerbox, address_from_maps for each agency
input_serper_addresses3sources = "/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/agency_data_preprocessedV2_Peilu.csv"
serper_address_3sources = pd.read_csv(input_serper_addresses3sources)

In [39]:
def fetch_serper_scraped_address_3sources(agency_name):
    if agency_name in serper_address_3sources["agency_name"].values: 
        return (serper_address_3sources.loc[serper_address_3sources["agency_name"] == agency_name, "address_from_search_snippet"].iloc[0],\
                serper_address_3sources.loc[serper_address_3sources["agency_name"] == agency_name, "address_from_search_answerbox"].iloc[0],\
                serper_address_3sources.loc[serper_address_3sources["agency_name"] == agency_name, "address_from_maps"].iloc[0])
                
    else: return ("NA", "NA", "NA")

In [40]:
afw_agency_address.assign(address_from_search_snippet = "NA")
afw_agency_address.assign(address_from_search_answerbox = "NA")
afw_agency_address.assign(address_from_maps = "NA")
for _, each_agency in afw_agency_address.iterrows():
    (address_from_search_snippet, address_from_search_answerbox,  address_from_maps) = fetch_serper_scraped_address_3sources(each_agency["agency_name"])
    afw_agency_address.loc[_, ["address_from_search_snippet"]] = address_from_search_snippet
    afw_agency_address.loc[_, ["address_from_search_answerbox"]] = address_from_search_answerbox
    afw_agency_address.loc[_, ["address_from_maps"]] = address_from_maps

/var/folders/70/3h59ks892kj0r487cwct1kzh0000gn/T/ipykernel_5427/2337406581.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1440 E Chapman Ave, Orange, CA 92866' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  afw_agency_address.loc[_, ["address_from_search_answerbox"]] = address_from_search_answerbox


In [41]:
print(afw_agency_address.head(2))
afw_agency_address.to_csv('/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/Returned_address_Web_scraping_AFWAddressCombined.csv', index = False)

     id                         agency_name             address1 address2  \
0  4778  Lucile Packard Children's Hospital         725 Welch Rd            
1  2297       Stanford Hospital and Clinics  900 Blake Wilbur Dr            

  site department       city state     zipcode               created_at  \
0                  Palo Alto    CA  94304-1601  2023-09-05 22:14:59.000   
1                  Palo Alto    CA  94304-2201  2024-02-06 21:02:15.000   

   year_creation  flight_count                           full_agency_address  \
0           2023           201         725 Welch Rd  Palo Alto CA 94304-1601   
1           2024            83  900 Blake Wilbur Dr  Palo Alto CA 94304-2201   

                 web_scraped_address  \
0  725 Welch Road Palo Alto CA 94304   
1                                NaN   

                         address_from_search_snippet  \
0  The phone number is (650) 497-8596. Our spirit...   
1  Contact us via phone number directory or onlin...   

  address_f

In [45]:
# Enable nested event loop support (needed for notebooks or embedded environments)
nest_asyncio.apply()

# Load API credentials
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT", "https://api.openai.com/v1/chat/completions")
MAX_CONCURRENT_REQUESTS = 10
RATE_LIMIT_RPM = 600
REQUEST_INTERVAL = 60 / RATE_LIMIT_RPM
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

async def analyze_address_match(full_agency_address: str, web_scraped_address: str) -> str:
    examples = """
Examples:
full_agency_address: "818 W Yakima Ave"
web_scraped_address: "818 W Yakima Ave, Yakima, WA 98902"
→ "Exact substring match"

full_agency_address: "1440 E Chapman Ave  Orange CA 92866-2221"
web_scraped_address: "1440 East Chapman Ave. Orange CA 92866"
→ "Exact substring match"

full_agency_address: "3-3420 Kuhio Hwy"
web_scraped_address: "3-3420 Kuhio Hwy, Lihue, HI 96766"
→ "Slight misspelling"

full_agency_address: "2021 Squalicum Pkwy"
web_scraped_address: "2021 Squalicium St #101, Nashua, NH 03060"
→ "One word difference"

full_agency_address: "111 South 11th Ave., Ste 120"
web_scraped_address: "11th Ave. 111 South, Ste 120"
→ "Same words, different order"

full_agency_address: "12 S 8th St"
web_scraped_address: "123 S 8th St"
→ "One word difference"

full_agency_address: "2801 N Gantenbein Ave  Portland OR 97227-1623"
web_scraped_address: "501 N. Graham Portland OR 97227"
→ "City, State, ZIPCODE are same"

full_agency_address: "314 S 11th Ave"
web_scraped_address: "402 S 12th Ave, Yakima, WA 98902"
→ "No match"

full_agency_address: "314 S 11th Ave"
web_scraped_address: ""
→ "One address missing"
"""

    prompt = f"""
Compare the two addresses and classify the match as one of the following:
1. "Exact substring match"
2. "Slight misspelling"
3. "Same words, different order"
4. "One word difference"
5. "City, State, ZIPCODE are same"
6. "No match"
7. "One address missing"

Respond ONLY with JSON like: {{ "match_category": "Exact substring match" }}

{examples}

Now compare:
full_agency_address: "{full_agency_address}"
web_scraped_address: "{web_scraped_address}"
"""

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }

    payload = {
        "model": OPENAI_MODEL,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that compares address similarity."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 100,
        "temperature": 0.1
    }

    async with semaphore:
        await asyncio.sleep(REQUEST_INTERVAL * (0.8 + 0.4 * random.random()))
        async with aiohttp.ClientSession() as session:
            async with session.post(OPENAI_ENDPOINT, headers=headers, json=payload) as response:
                if response.status == 200:
                    result = await response.json()
                    content = result['choices'][0]['message']['content']
                    try:
                        json_match = re.search(r'```json\s*(.*?)\s*```', content, re.DOTALL)
                        json_str = json_match.group(1) if json_match else content
                        json_str = re.sub(r'[^\x00-\x7F]+', '', json_str)
                        parsed = json.loads(json_str)
                        return parsed.get("match_category", "Parse error")
                    except Exception:
                        return "Parse error"
                return f"API error: {response.status}"

async def process_csv(input_path: str, output_path: str):
    with open(input_path, newline='', encoding='utf-8') as infile:
        reader = list(csv.DictReader(infile))
        fieldnames = list(reader[0].keys()) + ['match_category']

    results = []

    async def process_row(row):
        full_address = row.get("full_agency_address", "")
        web_address = row.get("web_scraped_address", "")
        row['match_category'] = await analyze_address_match(full_address, web_address)
        return row

    tasks = [process_row(row) for row in reader]
    processed = await asyncio.gather(*tasks)

    with open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(processed)

def run_address_match(input_csv):
    output_csv = f"/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/agency_address_compared_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    asyncio.run(process_csv(input_csv, output_csv))
    print(f"\nAddress matching completed. Output saved to: {output_csv}")

# To run this
run_address_match("/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/Returned_address_Web_scraping_AFWAddressCombined.csv")


Address matching completed. Output saved to: /Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/agency_address_compared_20250609_161342.csv


In [47]:
final_created_csv = pd.read_csv("/Users/shreya/Documents/UCDavis/Practicum/Spring_Quarter_Practicum/Web_Scraping_MyCodes/shreyaV2/agency_address_compared_20250609_161342.csv")

In [49]:
final_created_csv["match_category"].value_counts()

match_category
No match                         30
One address missing              19
Slight misspelling               11
Exact substring match             8
One word difference               6
City, State, ZIPCODE are same     1
Name: count, dtype: int64